<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/lecture-idea/60_linear_algebra_2/125_System_Linear_Eq_Five_Node_Truss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# 그래프, 수학 기능 추가
# Add graph and math features
import pylab as py
import numpy as np
import numpy.linalg as nl




# 가우스 소거법 : 선형 연립방정식의 풀이<br>Gauss Elimiation : Solving a System of Linear Equations



화살표를 그리는 함수<br>Function to draw an arrow



In [ ]:
def draw_2dvec(x, y, x0=0, y0=0, color='k', name=None):
    py.quiver(x0, y0, x, y, color=color, angles='xy', scale_units='xy', scale=1)
    py.plot((x0, x0+x), (y0, y0+y), alpha=0)
    if name is not None:
        if not name.startswith('$'):
            vec_str = '$\\vec{%s}$' % name
        else:
            vec_str = name
        py.text(0.5 * x + x0, 0.5 * y + y0, vec_str)



정삼각형을 그리는 함수<br>Function to draw an equilateral triangle



In [ ]:
def triangle_support(x, y, length):
    # https://matplotlib.org/gallery/lines_bars_and_markers/fill.html
    height = py.cos(py.radians(30)) * length
    py.fill((x, x + length*0.5, x + length*-0.5), (y, y - height, y - height))



## 5절점 트러스 예제<br>Example of a Five Node Truss



다음과 같은 트러스를 생각해 보자.<br>
Let's think about a truss as follows.<br>
(ref: "Application of system of linear equations", Chegg Study, [Online] Available:https://www.chegg.com/homework-help/questions-and-answers/application-system-linear-equations-sure-work-matlab-problem-figure-1-shows-mechanical-str-q22676917)



In [ ]:
# 마디점 좌표 nodal point coordinates
xy_list = [(0, 0), (1, 3**0.5), (2, 0), (3, 3**0.5), (4, 0)]

# 각 부재의 양 끝 점 end points of each member
connectivity_list = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (2, 4), (3, 4)]

for k, i_j in enumerate(connectivity_list):
    i, j = i_j

    py.plot(
        (xy_list[i][0], xy_list[j][0]),
        (xy_list[i][1], xy_list[j][1]),
        '.-'
    )

    # 부재 번호 표시 Indicate member id
    py.text(0.5 * (xy_list[i][0] + xy_list[j][0]), 
            0.5 * (xy_list[i][1] + xy_list[j][1]), k + 1)

# 마디점 번호 표시 Indicate node ids
for k, xy in enumerate(xy_list):
    py.text(xy[0], xy[1], '(%d)' % (k+1))
    
draw_2dvec(0, -0.5, xy_list[1][0], xy_list[1][1], name='$F_2$')
draw_2dvec(0, -0.5, xy_list[3][0], xy_list[3][1], name='$F_4$')

triangle_support(xy_list[0][0], xy_list[0][1], 0.25)
triangle_support(xy_list[4][0], xy_list[4][1], 0.25)

py.axis('equal')
# https://stackoverflow.com/questions/9295026/matplotlib-plots-removing-axis-legends-and-white-spaces
py.axis('off')

py.savefig('trapezoid_truss.svg')



모든 부재의 길이는 2m 이다.<br>
All members have length of 2m.



$F_2$ 과 $F_4$는 각각 $400N$ 과 $800N$ 이다.<br>
$F_2$ and $F_4$ are $400N$ and $800N$ respectively.



In [ ]:
F_2_N = 400
F_4_N = 800



우선, 전체 반력을 계산한다.<br>First of all, calculate the reaction force.



$$
\left\{
    \begin{align}
        R_{1x}+\sum F_x = 0 \\
        R_{1y}+R_{5y}+\sum F_y = 0 \\
        x_5 \cdot R_{5y}+\sum x \cdot F_y = 0 \\
    \end{align}
\right.
$$



알고 있는 상수를 오른쪽으로 옮기도록 하자.<br>Let's move known constants to the right side.



$$
\left\{
    \begin{align}
        R_{1x} &= 0 \\
        R_{1y}+R_{5y} &= - F_{2y} - F_{4y} \\
        x_5 \cdot R_{5y} &= - x_2 \cdot F_{2y} - x_5 \cdot F_{5y} \\
    \end{align}
\right.
$$



### 후진대입 예<br>An example of back substitution



#### 첫번째 해<br>First solution



$y$ 방향 반력에 관한 식만 살펴 보자.<br>Let's take a look at the equations about reaction forces in $y$ direction.



$$
\left\{
    \begin{align}
        1 \cdot R_{1y}+1 \cdot R_{5y} &= - F_{2y} - F_{4y} \\
        0 \cdot R_{1y}+x_5 \cdot R_{5y} &= - x_2 \cdot F_{2y} - x_5 \cdot F_{5y} \\
    \end{align}
\right.
$$



아래 방정식에서는 하나의 미지수는 계수가 0이고 다른 하나는 0이 아니다.<br>
In the lower equation, one unknown has coefficient 0 and the other non-zero.



여기서 계수가 0이 아닌 미지수는 해당 계수로 우변을 나누면 쉽게 알 수 있다.<br>Here, we can easily find the unknown with non-zero coefficient by dividing the right side with the coefficient.



$$
R_{5y} = \frac{1}{x_5} \left( - x_2 \cdot F_{2y} - x_5 \cdot F_{5y}  \right)
$$



이제 이 반력은 알려진 상수 이다.<br>Now this reaction force is a known constant.



다른 미지수를 구하기 위해 이를 이용할 수 있다.<br>We can use this to find other unknown(s).



#### 두번째 해<br>Second solution



첫번째 식을 살펴 보자. $R_{5y}$는 알고 있는 값이므로 오른쪽으로 옮긴다. <br>Let's look at the first equation. Because we know $R_{5y}$, let's move to the right side.



$$
1 \cdot R_{1y} = - 1 \cdot R_{5y} - F_{2y} - F_{4y}
$$



$$
R_{1y} =\frac{1}{1} \left(- 1 \cdot R_{5y} - F_{2y} - F_{4y}\right)
$$



이렇게 해서 모든 반력을 구할 수 있다.<br>This way, we can find all the reaction forces.



이렇게 먼저 알아낸 미지수 값을 이용하여 다른 미지수 값을 찾는 것을 **후진대입법**이라고 한다.<br> **Back subsitution** uses a solution found in advance to find other solutions.



Gauss 소거법은 이 후진대입법을 적용하기 좋은 형태로 계수 행렬을 변형시킨다.<br>
Gauss elimination would make the coefficient matrix suitable for the back substitution.



### 각 절점에서의 평형 방정식을 행렬 형태로 정리<br>Organizing equations of equilibrium at each node into a matrix form



트러스의 부재 사이의 각은 모두 60도이다.<br>The angles between the members are 60 degrees.



$$
\alpha=sin\left( \frac{\pi}{6} \right)=cos\left( \frac{\pi}{3} \right) \\
\beta=sin\left( \frac{\pi}{3} \right)=cos\left( \frac{\pi}{6} \right)
$$



#### (1) 번 마디<br>Node (1)



$x$ 방향 평형<br>
$x$ directional equilibrium.



$$
R_{1x} + \alpha f_1 + f_2 =0
$$



여기서 $f_i$는 부재 $i$ 번의 장력이다.<br>Here $f_i$ indicates the tensile force of $i$th member.



$y$ 방향 평형<br>
$y$ directional equilibrium.



$$
R_{1y} + \beta f_1 =0
$$



행렬 형태로 한번 정리해 보자.<br>Let's organize in a matrix form.



$$
\begin{bmatrix}
    1 & 0 & \alpha & 1 \\
    0 & 1 &  \beta & 0 \\
\end{bmatrix}
\begin{pmatrix}
    R_{1x} \\ R_{1y} \\ f_1 \\ f_2
\end{pmatrix}
=
\begin{pmatrix}
    0 \\ 0 \\ 0 \\ 0
\end{pmatrix}
$$



#### (2) 번 마디<br>Node (2)



(2)번 마디 $x$ 방향<br>
$x$ direction at node (2).



$$
-\alpha f_1 + \alpha f_3 + f_{4}=0
$$



(2)번 마디 $y$ 방향<br>
$y$ direction at node (2).



$$
-\beta f_1 - F_{2} -\beta f_3=0 \\
-\beta f_1 -\beta f_3=F_{2}
$$



$$
\begin{bmatrix}
    -\alpha & \alpha & 1 \\
     -\beta & -\beta & 0 \\
\end{bmatrix}
\begin{pmatrix}
    f_1 \\ f_3 \\ f_4
\end{pmatrix}
=
\begin{pmatrix}
    0 \\ F_2
\end{pmatrix}
$$



#### (3) 번 마디<br>Node (3)



$$
-f_2 -\alpha f_3 + \alpha f_5 + f_6 = 0 \\
\beta f_3 + \beta f_5 = 0
$$



$$
\begin{bmatrix}
   -1 & -\alpha & \alpha & 1 \\
    0 &   \beta &  \beta & 0 \\
\end{bmatrix}
\begin{pmatrix}
    f_2 \\ f_3 \\ f_5 \\ f_6
\end{pmatrix}
=
\begin{pmatrix}
    0 \\ 0
\end{pmatrix}
$$



#### (4) 번 마디<br>Node (4)



$$
-f_4 - \alpha f_5 + \alpha f_7 = 0 \\
- \beta f_5 - \beta f_7 = F_4 \\
$$



$$
\begin{bmatrix}
   -1 & -\alpha & \alpha \\
    0 &  -\beta & -\beta \\
\end{bmatrix}
\begin{pmatrix}
    f_4 \\ f_5 \\ f_7
\end{pmatrix}
=
\begin{pmatrix}
    0 \\ F_4
\end{pmatrix}
$$



#### (5) 번 마디<br>Node (5)



$$
-f_6 -\alpha f_7 = 0 \\
\beta f_7 + R_{5y} = 0
$$



$$
\begin{bmatrix}
   -1 & -\alpha & 0 \\
    0 &   \beta & 1 \\
\end{bmatrix}
\begin{pmatrix}
    f_6 \\ f_7 \\ R_{5y}
\end{pmatrix}
=
\begin{pmatrix}
    0 \\ 0
\end{pmatrix}
$$



![Truss](https://user-images.githubusercontent.com/17876446/246723126-436a9c8f-616b-4876-af73-0299eb78a8ea.svg)



행렬형태로는:<br>
In matrix form:



$$
\begin{bmatrix}
1 & 0 &  \alpha &  1 &       0 &  0 &       0 &  0 &       0 & 0\\
0 & 1 &   \beta &  0 &       0 &  0 &       0 &  0 &       0 & 0\\
0 & 0 & -\alpha &  0 &  \alpha &  1 &       0 &  0 &       0 & 0\\
0 & 0 &  -\beta &  0 &  -\beta &  0 &       0 &  0 &       0 & 0\\
0 & 0 &       0 & -1 & -\alpha &  0 &  \alpha &  1 &       0 & 0\\
0 & 0 &       0 &  0 &   \beta &  0 &   \beta &  0 &       0 & 0\\
0 & 0 &       0 &  0 &       0 & -1 & -\alpha &  0 &  \alpha & 0\\
0 & 0 &       0 &  0 &       0 &  0 &  -\beta &  0 &  -\beta & 0\\
0 & 0 &       0 &  0 &       0 &  0 &       0 & -1 & -\alpha & 0\\
0 & 0 &       0 &  0 &       0 &  0 &       0 &  0 &   \beta & 1\\
\end{bmatrix}
\begin{pmatrix}
R_{1x} \\ R_{1y} \\ f_1 \\ f_2 \\ f_3 \\ f_4 \\ f_5 \\ f_6 \\ f_7 \\ R_{5y}
\end{pmatrix}
=
\begin{pmatrix}
0 \\ 0 \\ 0 \\ F_2 \\ 0 \\ 0 \\ 0 \\ F_4 \\ 0 \\ 0
\end{pmatrix}
$$



주대각선 주위에 0이 아닌 요소들이 모여 있음을 알 수 있다.<br>
We can see that none-zero elements are close to the main diagonal.



행렬에 변수 값을 넣어 준비해 보자.<br>
Let's prepare a matrix with numerical values.



In [ ]:
alpha = py.sin(py.radians(30))
beta = py.sin(py.radians(60))



In [ ]:
matrix = np.array([
    [1, 0,  alpha,  1,      0,  0,      0,  0,      0, 0,],
    [0, 1,   beta,  0,      0,  0,      0,  0,      0, 0,],
    [0, 0, -alpha,  0,  alpha,  1,      0,  0,      0, 0,],
    [0, 0,  -beta,  0,  -beta,  0,      0,  0,      0, 0,],
    [0, 0,      0, -1, -alpha,  0,  alpha,  1,      0, 0,],
    [0, 0,      0,  0,   beta,  0,   beta,  0,      0, 0,],
    [0, 0,      0,  0,      0, -1, -alpha,  0,  alpha, 0,],
    [0, 0,      0,  0,      0,  0,  -beta,  0,  -beta, 0,],
    [0, 0,      0,  0,      0,  0,      0, -1, -alpha, 0,],
    [0, 0,      0,  0,      0,  0,      0,  0,   beta, 1,],
])



행렬의 계수를 계산해 보자.<br>Let's check the rank of the matrix.



In [ ]:
nl.matrix_rank(matrix)



미지수의 갯수와 정방행렬의 계수가 같다는 것은 이 선형 연립 방정식의 해를 구할 수 있다는 뜻이다.<br>
The number of unknowns and the rank of the matrix are the same; we can find a root of this system of linear equations.



우변을 준비해 보자.<br>
Let's prepare for the right side.



In [ ]:
vector = np.array([[0, 0, 0, F_2_N, 0, 0, 0, F_4_N, 0, 0,]]).T



파이썬의 확장 기능 가운데 하나인 NumPy 의 선형 대수 기능 `solve()` 를 사용하여 해를 구해 보자.<br>
Using `solve()` of linear algebra subpackage of `NumPy`, a Python package, let's find a solution.



In [ ]:
x = nl.solve(matrix, vector)
x



이렇게 찾은 `x`가 원래의 연립방정식을 만족하는지 확인해보자.<br>
Let's check whether `x` found this way satisfies the original system of equations.



In [ ]:
import numpy.testing as nt


nt.assert_allclose(matrix @ x, vector, atol=1e-7)



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

